In [1]:
import os
import shutil
import torch
from torchvision.datasets import ImageFolder

# === CONFIG ===
DATA_ROOT = r'../../IAM+RIMES'           # root del dataset usato da ImageFolder
SPLIT_PATH = r'splits/IAM+RIMES.pth'     # file split già creato (train_indices, test_indices, label_map)
OUT_ROOT = r'test_set'                   # cartella di destinazione

# Se vuoi ripulire ogni volta la cartella di output, metti True
CLEAN_OUTPUT = False

def main():
    # Carica dataset base (senza transform)
    dataset = ImageFolder(root=DATA_ROOT)

    # Carica split
    split = torch.load(SPLIT_PATH, map_location='cpu')
    test_indices = split['test_indices']
    label_map = split['label_map']        # dict {original_label_idx -> new_idx} SOLO per autorizzati
    authorized_label_ids = set(label_map.keys())

    # Mappa indice -> nome classe (writer)
    # In ImageFolder, dataset.classes[indice] = nome_cartella_writer
    idx_to_name = dataset.classes

    # Prepara cartelle di output
    if CLEAN_OUTPUT and os.path.isdir(OUT_ROOT):
        shutil.rmtree(OUT_ROOT)
    os.makedirs(os.path.join(OUT_ROOT, 'autorizzati'), exist_ok=True)
    os.makedirs(os.path.join(OUT_ROOT, 'non autorizzati'), exist_ok=True)

    # Conta
    count_auth = 0
    count_unauth = 0

    # ImageFolder salva (path, label) in dataset.samples con lo stesso ordine degli indici
    samples = dataset.samples  # lista di tuple (filepath, label)

    for idx in test_indices:
        img_path, orig_label = samples[idx]
        writer_name = idx_to_name[orig_label]

        group = 'autorizzati' if orig_label in authorized_label_ids else 'non autorizzati'
        if group == 'autorizzati':
            count_auth += 1
        else:
            count_unauth += 1

        # Crea cartella del writer
        dest_dir = os.path.join(OUT_ROOT, group, writer_name)
        os.makedirs(dest_dir, exist_ok=True)

        # Copia il file (mantiene nome originale)
        dest_path = os.path.join(dest_dir, os.path.basename(img_path))
        shutil.copy2(img_path, dest_path)

    print(f"✅ Creato '{OUT_ROOT}'")
    print(f" - Autorizzati: {count_auth} file in {len(os.listdir(os.path.join(OUT_ROOT, 'autorizzati')))} cartelle writer")
    print(f" - Non autorizzati: {count_unauth} file in {len(os.listdir(os.path.join(OUT_ROOT, 'non autorizzati')))} cartelle writer")

if __name__ == '__main__':
    main()


✅ Creato 'test_set'
 - Autorizzati: 100 file in 100 cartelle writer
 - Non autorizzati: 50 file in 50 cartelle writer
